In [1]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect_langs
import re

In [ ]:
#twitter api keys
auth = tweepy.OAuth1UserHandler(
   'ObDcu9DovwyYkx5w5tJouFGvP', '6A2UhUI9YpVis96xea1xmZqR5b2JIMIpUAOQIjJBaNPkINwnOj', '459226572-g5WB6p6QAcVciJ50uN6SLTRVfZKDqS0Bcp4bFGqd', 'jzFTq24nII5LUsG0FjsSCsrS0GLWN5ZK5BPIMfiJceUls')

api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
search_query = search_words + " -filter:retweets AND -filter:replies"

In [ ]:
limit = 100000
tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

In [ ]:
data=[]
for tweet in tweets:
    data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [ ]:
columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
bp_tweets=pd.DataFrame(data, columns=columns)
bp_tweets.to_csv("BlackPanther-24-11.csv")

In [2]:
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
30098,30098,1592942899159011328,GeekWatchOne,2022-11-16 18:09:17+00:00,"Of course, we're reviewing #BlackPanther #Waka...",Ohio,1,2,1653,1051,False,Twitter Web App
32801,32801,1592770765346439168,tweetticator,2022-11-16 06:45:17+00:00,Black women rulers in the history of the Yorub...,NaN,0,2,626,89,False,Twitter Web App
34238,34238,1592709251797028869,BTO353,2022-11-16 02:40:51+00:00,Stan lee the creator of #Blackpanther https://...,NaN,0,0,67,329,False,Twitter for iPhone
23556,23556,1593353475483701248,Josh_Johnson79,2022-11-17 21:20:46+00:00,NAMOR AINT PLAYIN #BlackPanther,"Irmo,SC",0,0,446,822,False,Twitter for iPhone
29533,29533,1592970801271214080,Disney_Insiders,2022-11-16 20:00:09+00:00,Return to Wakanda.\n\nMarvel Studios’ Black Pa...,NaN,56,785,76184,43,True,Sprinklr
28249,28249,1593028249076256770,Wheeljack83,2022-11-16 23:48:26+00:00,Delicious refreshments and pastries at today’s...,"Los Angeles, CA",0,2,471,323,False,Twitter for iPhone
16190,16190,1594008040071745537,david_jgpeoples,2022-11-19 16:41:46+00:00,I didn’t expect many surprises in #WakandaFore...,NaN,0,2,71,43,False,Twitter for iPhone
5609,5609,1594874420694245376,TheGoldinChyld,2022-11-22 02:04:27+00:00,#BlackPanther\n#TChalla\n#WakandaForever https...,NaN,0,0,179,809,False,Twitter for Android
8768,8768,1594520818817667077,kathia_woods,2022-11-21 02:39:22+00:00,No petition or hate shall keep us from the mon...,"Philadelphia, PA",2,14,4803,3884,False,Twitter for iPhone
27904,27904,1593046525516865536,gnurpiyani,2022-11-17 01:01:03+00:00,"Saya telah mengirimkan #ShopeePayTHR 🧧. Ayo, a...",NaN,0,0,618,398,False,Twitter for iPhone


In [ ]:
tweets.info()

In [ ]:
tweets.head()

In [15]:
tweets.drop(columns='Unnamed: 0', inplace = True)

In [17]:
tweets['location'].fillna('None', inplace=True)

In [21]:
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [10]:
def get_languages(tweet):
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [11]:
tweets['language'] = tweets['tweet'].apply(get_languages)

In [14]:
tweets.language.value_counts()[:10]

en    24188
es     3334
da     1493
pt     1456
fr      776
id      512
ja      473
th      469
no      377
it      330
Name: language, dtype: int64

In [71]:
def get_links(tweets):
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [72]:
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [73]:
tweets.contains_media.value_counts()

Yes    20872
No     14182
Name: contains_media, dtype: int64

In [77]:
tweets.sample(50)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,language,links,contains_media
28753,1593004189302804480,M_G_S409,2022-11-16 22:12:49,Rewatching #BlackPanther knowing what we know...,None,0,0,1400,1478,False,Twitter for iPhone,en,[],No
784,1595593942367625220,inMARVEL_,2022-11-24 01:43:35,Ryan Coogler lanza un mensaje de gratitud a lo...,Tierra 616,2,6,2221,203,False,Twitter for Android,es,"[(https://, https, t.co, t, co)]",Yes
4936,1594990572103688192,alx_africa,2022-11-22 09:46:00,#quote\n“How many times do I have to teach you...,"Nairobi, Kenya",27,79,16824,143,False,Twitter for Android,en,"[(https://, https, t.co, t, co)]",Yes
26065,1593176829480747009,PortalMarvelBR_,2022-11-17 09:38:50,Crítica de #WakandaForever :\n\nPontos posit...,None,0,3,185,91,False,Twitter for Android,pt,[],No
4749,1595031631127285760,FAliseyu,2022-11-22 12:29:09,Thanks @MRKINGSUREBETF2 for this your fixed ga...,"Rivers, Nigeria",1,2,74,145,False,Twitter for iPhone,en,"[(https://, https, t.co, t, co)]",Yes
5522,1594887086586613760,RobertoPeredaD,2022-11-22 02:54:47,Mi #wishlist\n\n#WakandaForever \n#AvatarTheWa...,"Aguascalientes, México",0,0,3998,975,False,Twitter for Android,en,[],No
7719,1594660005986586627,SHIELDZephyrOne,2022-11-21 11:52:27,#Namor Raises 3 Huge Questions About The #MCU’...,Location: Classified,0,2,732,4976,False,Twitter Web App,en,"[(https://, https, t.co, t, co)]",Yes
6506,1594778019683598341,DemenciaMedia,2022-11-21 19:41:24,#Novedades Marvel Studios presentó un nuevo pó...,None,1,4,45,61,False,Twitter for iPhone,es,"[(https://, https, t.co, t, co)]",Yes
33574,1592732813211881472,MapAnalyst,2022-11-16 04:14:28,This was a good ass movie. I love it. #Wakanda...,In the Cloud,0,2,319,981,False,Twitter for iPhone,en,[],No
13143,1594186865711624192,poliguirezende,2022-11-20 04:32:22,Será que eu chorei assistindo Wakanda Forever?...,Brasil,0,0,47,550,False,Twitter for Android,pt,"[(https://, https, t.co, t, co)]",Yes


In [ ]:
cv = tweets['location'].replace(np.nan, "None")

In [ ]:
cv[cv.str.contains('IL')][:60]
